Convert video into frame images

In [ ]:
import cv2
vidcap = cv2.VideoCapture('sample.mp4')
success,image = vidcap.read()
count = 1
while success:
  cv2.imwrite("sample/frame%d.jpg" % count, image)     # save frame as JPEG file
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

Crop Pedestrian in Bounding Box

In [ ]:
import pandas as pd
import cv2

df = pd.read_csv('sample.csv')
df[df < 0] = 0

for i in range(len(df)):
    frame = df.loc[i][0]
    img = cv2.imread('sample/frame'+str(frame)+'.jpg')
    ID = df.loc[i][1]
    x1 = df.loc[i][2]
    y1 = df.loc[i][3]
    x2 = df.loc[i][4]
    y2 = df.loc[i][5]
    # imgCrop = img[x2:x1, y2:y1]
    imgCrop = img[y1:y1+y2, x1:x2+x1]
    cv2.imwrite('sample_person/frame'+str(frame)+'_ID'+str(ID)+'.jpg',imgCrop)

Crop ROI

In [ ]:
import os
import cv2
directory = "Staff"
directory2 = "Staff_Cropped"


for filename in os.listdir(directory):
    if filename.endswith(".jpg"):
      #do smth
        img = cv2.imread(os.path.join(directory,filename))
        y = 75
        x = 40
        w = 60
        h = 50
        crop_img = img[y:y+h, x:x+w]
        filename2=filename.replace('.jpg', '')
        cv2.imwrite(os.path.join(directory2,filename2+"_crop.png"),crop_img)
        continue
    else:
        continue

Apply FAST and FASTNMS to the folder containing the ROI images and save the results in CSV file

In [ ]:
import os
import cv2
import csv

directory = "Staff_Final"

with open('Staff_fast.csv', mode='w',newline='') as Round:
    Round_writer = csv.writer(Round, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
      #do smt
            img = cv2.imread(os.path.join(directory,filename))
            # Convert the image to RGB
            rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Convert the image to gray scale
            gray = cv2.cvtColor(rgb,cv2.COLOR_BGR2GRAY)
            # ## Detect keypoints and Create Descriptor
            fast = cv2.FastFeatureDetector_create()
            # Detect keypoints with non max suppression
            keypoints_with_nonmax = fast.detect(gray, None)
            # Disable nonmaxSuppression
            fast.setNonmaxSuppression(False)
            # Detect keypoints without non max suppression
            keypoints_without_nonmax = fast.detect(gray, None)
            Round_writer.writerow([filename,len(keypoints_with_nonmax),len(keypoints_without_nonmax)])
            continue
        else:
            continue

SVM using FAST features stored in CSV file

In [ ]:
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('FAST_Combined.csv')
X = df.drop(['filename'], axis=1)
X=X.dropna()
X = X.apply(LabelEncoder().fit_transform)
y = pd.DataFrame(X, columns= ['class'])
X = pd.DataFrame(X, columns= ['FASTNMS'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
svclassifier = svm.SVC(kernel='linear')
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))